In [1]:
import numpy as np
import pandas as pd
import os
mapa_condiciones = {
    0: "normal",
    1: "tuberculosis",
    2: "neumonia",
    3: "covid-19"
}

In [ ]:
!pip install torchxrayvision torchio tqdm torchinfo

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
#%cd /content/drive/My\ Drive/ProyectoDIALUNG/
#Archivo local 
%cd /media/josejacomeb/Datos1/DatasetsMédicos/ProyectoDIALUNG/

/media/josejacomeb/Datos1/DatasetsMédicos/ProyectoDIALUNG


In [3]:
direccion_dataset_imagenes = "DIALUNG-Datasets/"
dataset_train_256 = os.path.join(direccion_dataset_imagenes, "train_256.hdf5") 
dataset_test_256 = os.path.join(direccion_dataset_imagenes, "test_256.hdf5")
dataset_val_256 = os.path.join(direccion_dataset_imagenes, "val_256.hdf5")

In [4]:
#Tamaño deseado de la imagen
ancho_imagen_deseado  = 224
alto_imagen_deseado =  224
#Variables de rotación
probabilidad_rotacion = 0.15
angulo_rotacion_grados = 25
#Variables de Blur
probabilidad_blur = 0.15
std_filtro_gaussiano = (1, 1)
#Variables de Ruido
probabilidad_ruido = 0.15
std_ruido_gaussiano = (0, 0.08)
mean_ruido_gaussiano = 0
# Variables de Desplazamiento
probabilidad_desplazamiento = 0.15
desplazamiento_x_y = (1, 1) #1 mm de desplazamiento en ambos ejes 
# Tamaño del batch
batch_size = 8

In [5]:
from torchvision.transforms import Compose #Compose me sirve para apilar varias transformaciones en una sola
from torchvision.transforms import Resize #Cargar herramienta para hacer Rescalado
from torchvision.transforms import Lambda #Convertir tensor de salida a one-hot-encoded 
import torch 
#Librerias de TorchIO
import torchio as tio
#Librerias de DIALUNG
from dialung.utils.creardatasetbinario import crearDatasetBinarioDIALUNG


resize_transform = tio.Resize((alto_imagen_deseado, ancho_imagen_deseado, 1)) #Cambiar el tamaño al establecido en la celda de arriba
normalizacion_dataset = tio.transforms.ZNormalization()
rescalar_intensidad = tio.transforms.RescaleIntensity(out_min_max=(0, 1)) #Rescalar 0 o 1
#Diccionario de transformaciones con su probabilidad de ser ejecutadas
diccionario_transformaciones = {
  tio.RandomAffine(degrees = angulo_rotacion_grados): probabilidad_rotacion, #transformacion de rotación  
  tio.transforms.RandomBlur(std = std_filtro_gaussiano): probabilidad_blur, #transformacion de blur  
  tio.transforms.RandomNoise(mean = mean_ruido_gaussiano, std = std_ruido_gaussiano): probabilidad_ruido, #transformaciones de ruido
  #tio.RandomAffine(translation = desplazamiento_x_y): probabilidad_desplazamiento #transformacion de traslación 
}
#Agrupar todas las transformaciones en una sola
transformaciones_DIALUNG = tio.OneOf(diccionario_transformaciones)

transformaciones_dataset = Compose([resize_transform, transformaciones_DIALUNG, rescalar_intensidad]) #Apila las transformaciones
transformacion_salida = Lambda(lambda y: torch.zeros(4, dtype=torch.float).scatter_(0, torch.tensor(int(y)), value=1)) #Hago un

#Creo objetos de tipo crearDatasetDIALUNG
train_datasetDIALUNG = crearDatasetBinarioDIALUNG(dataset_train_256, transformaciones_dataset, rgb = False)
test_datasetDIALUNG = crearDatasetBinarioDIALUNG(dataset_test_256, transformaciones_dataset, rgb = False)
val_datasetDIALUNG = crearDatasetBinarioDIALUNG(dataset_val_256, transformaciones_dataset, rgb = False)

In [6]:


from torch.utils.data import DataLoader #Clase dataloader para cargar en PyTorch los objetos de tipo Dataset

#Creo objetos de tipo DataLoader en lotes de 64 imagenes y barajeados aleatoriamente
train_dataloader = DataLoader(train_datasetDIALUNG, batch_size=batch_size, shuffle=True, num_workers = 2)
test_dataloader = DataLoader(test_datasetDIALUNG, batch_size=batch_size, shuffle=False, num_workers = 2)
val_dataloader = DataLoader(val_datasetDIALUNG, batch_size=batch_size, shuffle=False, num_workers = 2)



In [7]:
from torch import nn
from collections import OrderedDict
from datetime import date
from torch.utils.tensorboard import SummaryWriter

#Configuracion de datos de cada experimento
carpeta_experimentos = "DIALUNG_Experimentos"
carpeta_metricas, carpeta_modelos, carpeta_logs = "Métricas", "Modelos", "logs_dir"
nombre_modelo = "DenseNet121" #Poner el nombre cada uno de su modelo
implementado_por= "josejacomeb" #Poner sus nombres 
nombre_experimento = os.path.join(implementado_por, nombre_modelo)
ruta_guardar_metricas = os.path.join(carpeta_experimentos, carpeta_metricas, nombre_experimento)
ruta_guardar_modelos = os.path.join(carpeta_experimentos, carpeta_modelos, nombre_experimento)
ruta_guardar_logs = os.path.join(carpeta_experimentos, carpeta_logs, nombre_experimento)
#Creo los directorios
if not os.path.exists(ruta_guardar_metricas):
    os.makedirs(ruta_guardar_metricas)
if not os.path.exists(ruta_guardar_modelos):
    os.makedirs(ruta_guardar_modelos)
if not os.path.exists(ruta_guardar_logs):
    os.makedirs(ruta_guardar_logs)
#Inicializar TensorBoard para escribir los datos en el directorio de ruta_guardar_logs
writer = SummaryWriter(ruta_guardar_logs)

In [8]:
dispositivo = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(dispositivo))

Using cuda device


In [9]:
import torchxrayvision as xrv #Importar la librería TorchXRayVision

In [10]:
modelo = xrv.models.DenseNet(weights="densenet121-res224-all", num_classes=4)
in_features = modelo.classifier.in_features
modelo.classifier = nn.Linear(in_features=in_features, out_features=4) #Cambio la capa de salida para ajustar a los cuatro parámetros
modelo.op_threshs = None
modelo.to(dispositivo)

XRV-DenseNet121-densenet121-res224-all

In [11]:
# Obtener algunas imagenes para mostrar en TensorBoard
images, labels = next(iter(val_dataloader))
images = images.to(dispositivo)
#Añadir el modelo a tensorboard
writer.add_graph(modelo, images)

In [12]:
from torchinfo import summary 
print(summary(modelo, input_size = tuple(images.shape)))

Layer (type:depth-idx)                   Output Shape              Param #
DenseNet                                 --                        --
├─Sequential: 1-1                        [8, 1024, 7, 7]           --
│    └─Conv2d: 2-1                       [8, 64, 112, 112]         3,136
│    └─BatchNorm2d: 2-2                  [8, 64, 112, 112]         128
│    └─ReLU: 2-3                         [8, 64, 112, 112]         --
│    └─MaxPool2d: 2-4                    [8, 64, 56, 56]           --
│    └─_DenseBlock: 2-5                  [8, 256, 56, 56]          --
│    │    └─_DenseLayer: 3-1             [8, 96, 56, 56]           45,440
│    │    └─_DenseLayer: 3-2             [8, 128, 56, 56]          49,600
│    │    └─_DenseLayer: 3-3             [8, 160, 56, 56]          53,760
│    │    └─_DenseLayer: 3-4             [8, 192, 56, 56]          57,920
│    │    └─_DenseLayer: 3-5             [8, 224, 56, 56]          62,080
│    │    └─_DenseLayer: 3-6             [8, 256, 56, 56]    

In [13]:
learning_rate = 1e-3
epochs = 50
momentum = 0 #Momento del algoritmo SGD

In [14]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
funcion_perdida = nn.CrossEntropyLoss() #Uso de CrossEntropy
optimizador = torch.optim.SGD(modelo.parameters(), lr=learning_rate, momentum=momentum) #Optimizador SGD
scheduler = ReduceLROnPlateau(optimizador, "min", patience = 5, factor = 0.60, min_lr = 1e-5)

In [18]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir $ruta_guardar_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [17]:
from dialung.entrenarvalidar import EntrenamientoValidacionDIALUNG

ruta_ultimo_archivo_pesos = os.path.join(ruta_guardar_modelos, "ultimo.pt") #mejor.pt si algo falló en el proceso
epoch_inicio = 0
resumir_entrenamiento = False
step_inicio = 0
historico = None

if os.path.exists(ruta_ultimo_archivo_pesos):
    resumir_entrenamiento = True
    checkpoint = torch.load(ruta_ultimo_archivo_pesos)
    modelo.load_state_dict(checkpoint["model_state_dict"])#cargar los pesos del modelo
    optimizador.load_state_dict(checkpoint["optimizer_state_dict"]) #carga el modelo del optimizador
    #Checar por una nuevo lr seleccionado por el usuario
    if learning_rate != optimizador.param_groups[0]["lr"]:
        optimizador.param_groups[0]["lr"] = learning_rate
    if momentum != optimizador.param_groups[0]["momentum"]:
        optimizador.param_groups[0]["momentum"] = momentum
    
    epoch_inicio = checkpoint["epoch"]
    step_inicio = checkpoint["steps"]
    historico = checkpoint["historico"]
    print("Resumiendo el entrenamiento desde la época: {}".format(epoch_inicio))
else:
    print("No hay un checkpoint guardado, iniciando el entrenamiento desde la época 0")

ent_val_DIALUNG = EntrenamientoValidacionDIALUNG(modelo = modelo, funcion_perdida = funcion_perdida,
                                                 dispositivo = dispositivo, optimizador = optimizador, 
                                                 scheduler = scheduler, ruta_writer=ruta_guardar_logs)
if resumir_entrenamiento:
    ent_val_DIALUNG.entrenamiento_global_step = step_inicio
    ent_val_DIALUNG.H = historico 
    ent_val_DIALUNG.mejor_precision = max(historico["val_acc"]) #Cargo la mejor perdida del modelo

for epoch in range(epoch_inicio, epochs):
    print(f"Época {epoch+1}\n-------------------------------")
    ent_val_DIALUNG.entrenar(train_dataloader)
    ent_val_DIALUNG.validar(test_dataloader)
    ent_val_DIALUNG.guardar_modelo(ruta_modelo=ruta_guardar_modelos, #Guardar modelo y Métricas
                                   ruta_metricas=ruta_guardar_metricas, epoch=epoch)
print("Entrenamiento finalizado!")

Resumiendo el entrenamiento desde la época: 0
Época 1
-------------------------------
## Entrenamiento ##


Progreso dataset entrenamiento(batches):   0%|          | 0/50 [00:00<?, ?it/s]

Valor Función pérdida: 1.392 Precisión: 38%  Tasa de aprendizaje: 0.001
Tiempo de iteración: 0:00:26.385391
## Validación ##


Progreso dataset evaluacion (Batches):   0%|          | 0/10 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      normal       0.36      0.92      0.52        26
tuberculosis       0.30      0.20      0.24        15
    neumonia       0.00      0.00      0.00        22
    covid-19       0.00      0.00      0.00        14

    accuracy                           0.35        77
   macro avg       0.16      0.28      0.19        77
weighted avg       0.18      0.35      0.22        77

Valor Función pérdida: 1.384 Precisión: 35 %
Tiempo de iteración: 0:00:01.800869
Época 2
-------------------------------
## Entrenamiento ##


...

Valor Función pérdida: 1.074 Precisión: 63%  Tasa de aprendizaje: 0.001
Tiempo de iteración: 0:00:26.281391
## Validación ##


Progreso dataset evaluacion (Batches):   0%|          | 0/10 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      normal       0.55      1.00      0.71        26
tuberculosis       1.00      0.07      0.12        15
    neumonia       0.95      0.86      0.90        22
    covid-19       0.67      0.43      0.52        14

    accuracy                           0.68        77
   macro avg       0.79      0.59      0.57        77
weighted avg       0.77      0.68      0.62        77

Valor Función pérdida: 1.059 Precisión: 68 %
Tiempo de iteración: 0:00:01.895555
Época 49
-------------------------------
## Entrenamiento ##


Progreso dataset entrenamiento(batches):   0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()